In [0]:
%sql
USE CATALOG de_project_catalog;
USE SCHEMA olist_silver;

In [0]:
Bronze_base_path = "/Volumes/de_project_catalog/olist_bronze/raw_data/bronze"
Silver_base_path = "/Volumes/de_project_catalog/olist_silver/cleaned_data"

### Orders

In [0]:
bronze_orders_df = spark.read.parquet(
   f"{Bronze_base_path}/orders"
)

bronze_orders_df.printSchema()
bronze_orders_df.show(5)

In [0]:
bronze_orders_df.groupBy("order_id").count().filter("count > 1").count()

In [0]:
bronze_orders_df.display()

In [0]:
# deduplicating the data and making sure order_id not null
from pyspark.sql.functions import row_number, desc
from pyspark.sql.window import Window
orders_clean_df =(
     bronze_orders_df
     .filter("order_id is Not Null")
     .withColumn("rn", row_number().over(Window.partitionBy("order_id").orderBy(desc("order_purchase_timestamp"))))
     .filter("rn == 1")
     .drop("rn", "corrupt_data")
)
orders_clean_df.display()

In [0]:
## customer_id and order_purchase_timestamp cannot be null as these two are mandatory columns
orders_clean_df = orders_clean_df.filter(
     "customer_id IS NOT NULL AND  order_purchase_timestamp IS NOT NULL"
)

In [0]:
#standardizing the order status
orders_clean_df.select("order_status").distinct().show()

In [0]:
from pyspark.sql.functions import lower, trim
orders_clean_df = orders_clean_df.withColumn(
    "order_status",
    lower(trim("order_status"))
)

In [0]:
#extracting year and month
from pyspark.sql.functions import year, month
orders_clean_df = orders_clean_df.withColumn("order_year", year("order_purchase_timestamp"))\
          .withColumn("order_month", year("order_purchase_timestamp"))

In [0]:
orders_clean_df.show(5)

#### Data Validations

In [0]:
# Distinct Order_status
orders_clean_df.select("order_status").distinct().show()

In [0]:
# Total Order count
from pyspark.sql.functions import count
orders_clean_df.selectExpr("count(*) as tot_count").show()

In [0]:
# Null Check on Critical Columns
orders_clean_df.filter("order_purchase_timestamp IS NULL").count()

In [0]:
#sample data check
orders_clean_df.select(
     "order_id",
     "order_status",
     "order_purchase_timestamp",
    "order_year"
).show(5, truncate = False)

In [0]:
#writing clean orders to silver layer folder
orders_clean_df.write\
    .mode("overwrite")\
        .partitionBy("order_year")\
            .parquet(f"{Silver_base_path}/orders")